In [1]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpAffineExpression, \
LpConstraint, PULP_CBC_CMD
import numpy as np
import pulp as plp

In [2]:
foods = pd.read_csv('./Data_formatted/nv_df.csv', index_col = 'Unnamed: 0')
foods.index = [food.replace('-','_') for food in foods.index]
constraints = pd.read_csv('./Data_formatted/constraints.csv', 
                          index_col = 'Unnamed: 0')
constraints = constraints[1:] # remove calories
# Adjust amino acid requirements for bodyweight
usr_mass = input("How many kilos do you weigh?")
usr_mass = int(usr_mass)
constraints.iloc[-11:,0] *= usr_mass

FileNotFoundError: [Errno 2] No such file or directory: './Data_formatted/nv_df.csv'

In [ ]:
def formulateMP(foods, constraints):
    '''This function will take two dataframes, one foods*nutrients, the
    other constraints*bounds and return the PuLP lp formulation for minimizing
    calories. '''
    
    # Decision Variables
    food_vars  = {food:
    plp.LpVariable(cat=plp.LpContinuous, 
                   lowBound=0,
                   upBound=5,
                   name=f"{food}") 
    for food in foods.index.values}

    # Objective Function
    min_cals = plp.LpProblem("Min_Calories",plp.LpMinimize)
    min_cals += lpSum([foods.calories[food]*food_vars[food] 
                        for food in foods.index.values])
    # Add constraints
    for c in constraints.index:
        if not np.isnan(constraints.loc[c][0]):
            min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                              for food in foods.index]),
                                 sense=plp.LpConstraintGE, 
                                 rhs=constraints.loc[c][0],
                                 name=f'min_{c}'
                                )
        if not np.isnan(constraints.loc[c][1]):
            min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                              for food in foods.index]),
                                 sense=plp.LpConstraintLE, 
                                 rhs=constraints.loc[c][1],
                                 name=f'max_{c}'
                                )
    
    return min_cals

In [ ]:
lp_formulation = formulateMP(foods,constraints)

In [ ]:
lp_formulation.solve()

In [ ]:
def checkFeasibility(foods, constraints):
    '''Formulate lp problem and return status and optimal coefficients if 
    feasible as 2-tuple'''
    
    min_cals = formulateMP(foods,constraints)
    min_cals.solve(PULP_CBC_CMD(msg=0))
    return min_cals.status, min_cals.coefficients

In [ ]:
def infeasibilitySearch(foods, constraints):
    '''return 2d array with indices of constraints that constitute a solvable model
    and second the indices which cause an insolvable model when included.'''
    
    solvable = []
    brk_pts = []
    for i in range(0,constraints.shape[0]):
        candidates = [i for i in range(0,i+1) if i not in brk_pts]
        status = checkFeasibility(foods,constraints.iloc[candidates,:])[0]
        if (status == -1):
            brk_pts.append(i)
        else:
            solvable.append(i)
    return [solvable,brk_pts]

In [ ]:
infeasibilitySearch(foods,constraints)

In [ ]:
constraints.iloc[[18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]

Constraints with indices [18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39] are not solvable when added to the formulation.  These constraints are: \
'copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine',
       'histidine', 'leucine', 'lysine', 'methionine',
       'phenylalanine_tyrosine', 'threonine', 'valine' \
\
It is interesting that many of these are amino acids.  



###### What do the loadings look like for the solution with the infeasible constraints removed?

In [ ]:
i_c = [18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
feasible = [i for i in range(constraints.shape[0]) if i not in i_c]
f_c = constraints.iloc[feasible]

In [ ]:
model = formulateMP(foods,f_c)

In [ ]:
model.solve()

In [ ]:
model.coefficients()

In [ ]:
total_cals = 0
food_servings = {}
for v in model.variables():
    total_cals += v.varValue*foods.at[v.name,'calories']
    food_servings[v.name] = v.varValue

In [ ]:
food_servings[v.name] = v.varValue

In [ ]:
{food:srv for food, srv in food_servings.items() if srv != 0}

In [ ]:
{food:srv for food, srv in food_servings if srv != 0}

In [ ]:
total_cals

In [ ]:
sorted(foods.index.values)

In [ ]:
servings = [food[2] for food in model.coefficients()][0:foods.shape[0]]

In [ ]:
# 109,835 calories under constraints -- this result requires investigation.  
sum(servings*foods.calories)

In [ ]:
[c for c in range(0,40) if c not in candidates ]

In [ ]:
constraints.index.values[[c for c in range(0,40) if c not in candidates ]]

In [ ]:
# narrowing the constraints to non-macros, which may have been setting a floor on
# the total number of calories that limits optimization

macros = ['protein', 'fat_total', 'saturated_fat', 'carbohydrates']
constraints_micros_aa = constraints.loc[[c for c in constraints.index.values if 
                                         c not in macros]]

'copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine', 'histidine', 'leucine', 'lysine', 'methionine', 'phenylalanine_tyrosine', and 'threonine' are not solvable when all constraints are used in search.

In [ ]:
vitamins = ['choline', 'folate', 'niacin', 'pantothenic_acid', 'vitamin_b2',
       'vitamin_b1', 'vitamin_a', 'vitamin_b12', 'vitamin_b6', 'vitamin_c',
       'vitamin_d_mcg', 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium',
       'manganese', 'phosphorus', 'potassium', 'selenium', 'zinc']

for v in vitamins[0:5]:
    if not np.isnan(constraints.loc[v][0]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,v]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintGE, 
                             rhs=constraints.loc[v][0],
                             name=f'min_{v}'
                            )
    if not np.isnan(constraints.loc[v][1]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,v]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintLE, 
                             rhs=constraints.loc[v][1],
                             name=f'max_{v}'
                            )

In [ ]:
min_cals.solve()
min_cals.objective.value()

In [ ]:
for c in constraints.index[]:
    if not np.isnan(constraints.loc[c][0]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintGE, 
                             rhs=constraints.loc[c][0],
                             name=f'min_{c}'
                            )
    if not np.isnan(constraints.loc[c][1]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintLE, 
                             rhs=constraints.loc[c][1],
                             name=f'max_{c}'
                            )

In [ ]:
core_constraints = ['protein', 'cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine']

In [ ]:
constraints.index

In [ ]:
min_cals.solve()

In [ ]:
dir(plp);

In [ ]:
# Constraints

plp.Lp

In [ ]:
pulp.lpSum

In [ ]:
protein_calories = foods['protein']/foods['calories'].sort_values(ascending=False).copy()

In [ ]:
protein_calories.sort_values(ascending=False)

In [ ]:
# selecting high protein to calorie ratio foods
protein_foods = foods.loc[protein_calories.sort_values(ascending=False)[0:40].index,:]\
.query("calories > 50")

In [ ]:
constraints.drop(['saturated_fat','cholesterol'])

In [ ]:
necessary_nutrients = [c for c in constraints.index if c not in ['saturated_fat','cholesterol']]

In [ ]:
constraints.loc[~constraints.index.isin(['saturated_fat','cholesterol']),:].index.values

In [ ]:
foods[necessary_nutrients].idxmax()

In [ ]:
max_foods = foods.loc[list(set(foods[necessary_nutrients].idxmax().values)),:]

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in max_foods.index],
}

data_foods.update({col: protein_foods[col] for col in protein_foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Integer')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Next I will solve to minimize calories since minimizing for serving size has no solution.

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Looking to minimize calories:

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

In [ ]:
constraints.drop(index=['calories'],inplace=True)
# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")